![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important.

For this project, you're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

**Instructions:**
Analyze the manufacturing_parts table and determine whether a manufacturing process is working well or requires adjustment:

Create an alert that flags whether the height of a product is within the control limits for each operator using the formulas provided in the notebook. The final query should return the following fields: _operator, row_number, height, avg_height, stddev_height, ucl, lcl, alert_, and be ordered by the the item_no. Use a window function of length 5 considering rows before and including the current row; incomplete window rows should be removed in the final query output. Save this DataFrame as alerts.

In [17]:
WITH statistics AS (
    SELECT operator,
           ROW_NUMBER() OVER (ORDER BY item_no) AS row_number, 
           height, 
           AVG(height) OVER (PARTITION BY operator ORDER BY item_no ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS avg_height,
           STDDEV(height) OVER (PARTITION BY operator ORDER BY item_no ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS stddev_height,
           item_no
    FROM manufacturing_parts
)
SELECT operator, row_number, height, avg_height, stddev_height, ucl, lcl,
           CASE 
               WHEN height > ucl OR height < lcl THEN TRUE
               ELSE FALSE
           END AS alert
FROM (   
        SELECT operator, row_number, height, avg_height, stddev_height, 
               (avg_height + 3 * stddev_height / SQRT(5)) AS ucl,
               (avg_height - 3 * stddev_height / SQRT(5)) AS lcl, 
               item_no
        FROM statistics
        WHERE row_number > 5 AND row_number < 491
    ) AS subquery
;

operator  row_number  height  ...        ucl        lcl  alert
0       Op-1           6   20.36  ...  21.375477  18.448523  False
1       Op-1           7   20.22  ...  21.485108  18.574892  False
2       Op-1           8   21.03  ...  21.183369  18.684631  False
3       Op-1           9   19.78  ...  20.973418  19.366582  False
4       Op-1          10   20.71  ...  21.059711  19.780289  False
..       ...         ...     ...  ...        ...        ...    ...
480    Op-20         486   20.08  ...  20.021096  18.722904   True
481    Op-20         487   19.83  ...  20.177947  18.854053  False
482    Op-20         488   19.74  ...  20.200552  18.859448  False
483    Op-20         489   19.48  ...  20.182960  19.061040  False
484    Op-20         490   21.38  ...  21.102778  19.101222   True

[485 rows x 8 columns]


# EXPLANATION

## Common Table Expression (CTE) - statistics:
In order to avoid many nested subqueries, a CTE has been used. The statistics CTE calculates the moving average (avg_height) and standard deviation (stddev_height) of the height column within a window of the current row and the preceding four rows, partitioned by the operator column and ordered by item_no.
Notice that some moving averages and std dev might be incomplete: the first 4 rows and the last 4 rows.This will be corrected later in a subquery.

## Subquery:

The subquery in the FROM statement of the main query is used to calculate the control limits (ucl and lcl) for each row.
The WHERE clause filters out rows where the row number is less than 5 or greater than or equal to 491. This is done to exclude the incomplete windows.

## Main Query:

The main query selects columns from the subquery and computes the alert column.
It uses a CASE statement to determine whether an alert should be triggered based on the height being outside the control limits (ucl and lcl).
If the height is between the admisible limits then is TRUE; otherwise, it sets it to FALSE.


# COMMENTS

This project is interesting because combines window functions, subqueries, CTE and conditionals (CASE WHEN) it simulates a real 